In [1]:
library(Seurat)
library(dplyr)
library(ggplot2)
library(Signac)
library(GenomeInfoDb)
library(BSgenome.Hsapiens.UCSC.hg38)
library(EnsDb.Hsapiens.v86) # for a set of annotations for hg38
library(GPTCelltype)
library(openai)

mergedobj = readRDS("/volume/multiomeproj/combined_multiome_postUCSF.rds")


The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)

Attaching SeuratObject


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘pa

In [6]:
Idents(mergedobj) <- 'seurat_clusters'
suppressWarnings({
  all.markers <- FindAllMarkers(object = mergedobj)
})

Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5

Calculating cluster 6

Calculating cluster 7

Calculating cluster 8

Calculating cluster 9

Calculating cluster 10

Calculating cluster 11

Calculating cluster 12

Calculating cluster 13

Calculating cluster 14

Calculating cluster 15

Calculating cluster 16

Calculating cluster 17

Calculating cluster 18

Calculating cluster 19

Calculating cluster 20



In [7]:
Sys.setenv(OPENAI_API_KEY = '')

res <- gptcelltype(all.markers, 
            tissuename = 'human PBMC', 
            model = 'gpt-4'
)

cat(res)


[1] "Note: OpenAI API key not found: returning the prompt itself."
Identify cell types of human PBMC cells using the following markers separately for each
 row. Only provide the cell type name. Do not show numbers before the name.
 Some can be a mixture of multiple cell types. 
0:IL1B,ACSL1,PLAUR,NAMPT,CSF3R,FCAR,VCAN,PID1,THBS1,DMXL2
1:BNC2,GNLY,CD247,SYNE1,CCL5,PRKCH,CD44,FOXP1,GNAQ,UBE2E2
2:NLRP3,IL1B,ACSL1,VCAN,NAMPT,LRMDA,PLAUR,FCAR,JARID2,PID1
3:LEF1,TSHZ2,NELL2,FHIT,PRKCA,CAMK4,TXK,NDFIP1,BCL2,MAML2
4:INPP4B,ANK3,BCL11B,ARHGAP15,ARHGAP26,AOAH,CD83,ZEB2,LYN,RORA
5:BANK1,AFF3,EBF1,RALGPS2,COL19A1,FCRL1,PAX5,MS4A1,OSBPL10,BLK
6:LYN,NAMPT,LRMDA,PLXDC2,NEAT1,UBE2E2,FYN,RBM47,CD83,GRK3
7:TCF7L2,FMNL2,TBC1D8,MTSS1,SLC8A1,TMTC1,CD86,PECAM1,WARS,LYN
8:VCAN,CLEC7A,LRMDA,CSF3R,PLXDC2,DMXL2,NAMPT,NEAT1,ARHGAP26,RBM47
9:AC020916.1,SLC11A1,MALAT1,CNTNAP2,RRP12,MYO1F,RBM47,SAMHD1,NAMPT,AC010186.3
10:CCSER1,CST3,NEGR1,FLT3,FCER1A,DTNA,SH3BP4,HLA-DRA,ST18,HLA-DRB1
11:CCL5,TRDC,AC243829.2,NKG7,GN

# OUTPUT OF GPT 4 USING COPILOT

0: Classical Monocytes 
1: Effector Memory CD8+ T cells 
2: Non-classical Monocytes 
3: Central Memory CD4+ T cells 
4: Memory B cells 
5: Naive B cells 
6: Activated B cells / Classical Monocytes 
7: Vascular Endothelial cells 
8: Classical Monocytes / Myeloid Dendritic cells 
9: Classical Monocytes 
10: Myeloid Dendritic cells 
11: Effector Memory CD8+ T cells 
12: Effector Memory CD8+ T cells 
13: Mature Natural Killer cells 
14: Activated Plasma cells 
15: Cortical Neurons 
16: Effector Memory CD8+ T cells 
17: Activated Platelets 
18: Activated B cells / Classical Monocytes 
19: Mitochondria (not a cell type, but these are mitochondrial genes) 
20: Cortical Neurons



In [2]:
# MAP CLUSTER ANNOTATIONS
mapping <- c("0" = "Classical Monocytes",
             "1" = "Effector Memory CD8+ T cells",
             "2" = "Non-classical Monocytes",
             "3" = "Central Memory CD4+ T cells",
             "4" = "Memory B cells",
             "5" = "Naïve B cells",
             "6" = "Memory B cells",
             "7" = "Vascular Endothelial cells",
             "8" = "Classical Monocytes",
             "9" = "Classical Monocytes",
             "10" = "Plasmacytoid Dendritic cells",
             "11" = "Effector Memory CD8+ T cells",
             "12" = "Effector Memory CD8+ T cells",
             "13" = "Mature Natural Killer cells",
             "14" = "Activated Plasma cells",
             "15" = "Cortical Neurons",
             "16" = "Effector Memory CD8+ T cells",
             "17" = "Platelets",
             "18" = "Memory B cells",
             "19" = "Mitochondrial genes",
             "20" = "Cortical Neurons")
mergedobj@meta.data$gpt4_annotations <- mapping[as.character(mergedobj@meta.data$seurat_clusters)]

pdf("/volume/multiomeproj/gpt4annotations.pdf")
    print(DimPlot(mergedobj, group.by = 'gpt4_annotations', reduction = 'umap.harmony.wnn', label = TRUE, repel = TRUE))
dev.off()



png 
  2